In [8]:
review_cliente = """Este soprador de folhas é bastante incrível. Ele tem 
quatro configurações: sopro de vela, brisa suave, cidade ventosa 
e tornado. Chegou em dois dias, bem a tempo para o presente de 
aniversário da minha esposa. Acho que minha esposa gostou tanto 
que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho 
usado em todas as manhãs alternadas para limpar as folhas do 
nosso gramado. É um pouco mais caro do que os outros sopradores 
de folhas disponíveis no mercado, mas acho que vale a pena pelas 
características extras."""

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

schema_presente = ResponseSchema(
    name="presente", 
    type="bool",
    description="O  item foi comprado para alguem ? True caso verdadeiro e False caso falso ou não tenha a informação "
    )

schema_entrega = ResponseSchema(
    name="dias_entrega",
    type="int",
    description=" Quantos dias para a entrega chegar ?  se a resposta não for encontrada, retorne -1"
    )

schema_valor = ResponseSchema(
    name="percepcao_de_valor",
    type="list",
    description=" Extrai qualquer frase sobre o valor ou preço. Retorne como uma lista de Pyton."
    )

response_schema = [
    schema_presente,
    schema_entrega,
    schema_valor
] 

output_parser = StructuredOutputParser.from_response_schemas(response_schema)

schema_formatado = output_parser.get_format_instructions()

print(schema_formatado)

In [ ]:
from langchain.prompts import ChatPromptTemplate

review_template = ChatPromptTemplate.from_template("""
Para o texto a seguir, extraia as seguintes informaçoes:
presente, dias_entrega, percepcao_de_valor.

Texto:{review}

{schema}
""", partial_variables={"schema": schema_formatado})

print(review_template.format_messages(review=review_cliente))

In [14]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

resposta = chat.invoke(review_template.format_messages(review = review_cliente))

print(resposta.content)

```json
{
	"presente": true,
	"dias_entrega": 2,
	"percepcao_de_valor": ["É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."]
}
```


In [18]:
respota_json = output_parser.parse(resposta.content)

print(respota_json['presente'])
print(respota_json['dias_entrega'])
print(respota_json['percepcao_de_valor'])

True
2
['É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras.']
